In [1]:
import probtorch
import torch

import combinators
import smc
import ssm
import utils

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [4]:
for t in range(zs.shape[1] - 1):
    zs, mu, sigma, delta, generative = ssm.ssm_step(zs, mu, sigma, delta, t+1, generative)

In [5]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Normal([torch.FloatTensor of size 1x1]), 'X_

In [6]:
num_particles = 500

In [7]:
smc_step = smc.smc(ssm.ssm_step, ssm.ssm_retrace)

In [8]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [9]:
inference, init_ssm = smc.variational_smc(num_particles, ssm.init_ssm, smc_step, 500, 50, ssm_params, generative)

In [10]:
inference

Trace{'mu': Normal([torch.cuda.FloatTensor of size 500]), 'sigma': Normal([torch.cuda.FloatTensor of size 500]), 'delta': Normal([torch.cuda.FloatTensor of size 500]), 'Z_0': Normal([torch.cuda.FloatTensor of size 500]), 'Z_1': Normal([torch.cuda.FloatTensor of size 500]), 'X_1': Normal([torch.cuda.FloatTensor of size 500x1]), 'Z_2': Normal([torch.cuda.FloatTensor of size 500]), 'X_2': Normal([torch.cuda.FloatTensor of size 500x1]), 'Z_3': Normal([torch.cuda.FloatTensor of size 500]), 'X_3': Normal([torch.cuda.FloatTensor of size 500x1]), 'Z_4': Normal([torch.cuda.FloatTensor of size 500]), 'X_4': Normal([torch.cuda.FloatTensor of size 500x1]), 'Z_5': Normal([torch.cuda.FloatTensor of size 500]), 'X_5': Normal([torch.cuda.FloatTensor of size 500x1]), 'Z_6': Normal([torch.cuda.FloatTensor of size 500]), 'X_6': Normal([torch.cuda.FloatTensor of size 500x1]), 'Z_7': Normal([torch.cuda.FloatTensor of size 500]), 'X_7': Normal([torch.cuda.FloatTensor of size 500x1]), 'Z_8': Normal([torch.cu

In [11]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Normal([torch.FloatTensor of size 1x1]), 'X_

In [12]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 616.224731
SMC MSE at time 2: 220.069305
SMC MSE at time 3: 12.920108
SMC MSE at time 4: 2.852032
SMC MSE at time 5: 475.883667
SMC MSE at time 6: 308.648041
SMC MSE at time 7: 555.606018
SMC MSE at time 8: 47.613407
SMC MSE at time 9: 2225.171631
SMC MSE at time 10: 410.863403
SMC MSE at time 11: 28.431664
SMC MSE at time 12: 340.140656
SMC MSE at time 13: 7349.819824
SMC MSE at time 14: 4502.643555
SMC MSE at time 15: 257.731171
SMC MSE at time 16: 82.166145
SMC MSE at time 17: 2900.500244
SMC MSE at time 18: 8208.047852
SMC MSE at time 19: 3260.217529
SMC MSE at time 20: 3521.967773
SMC MSE at time 21: 1891.838867
SMC MSE at time 22: 14566.572266
SMC MSE at time 23: 318.286743
SMC MSE at time 24: 675.926819
SMC MSE at time 25: 4827.039062
SMC MSE at time 26: 5462.046875
SMC MSE at time 27: 2702.441406
SMC MSE at time 28: 2855.416748
SMC MSE at time 29: 12459.928711
SMC MSE at time 30: 9374.930664
SMC MSE at time 31: 21338.238281
SMC MSE at time 32: 14878.122070
SM